In [87]:
import pandas as pd
import numpy as np

In [88]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shivamb/netflix-shows")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\vinit\.cache\kagglehub\datasets\shivamb\netflix-shows\versions\5


In [89]:
df = pd.read_csv(path +"/netflix_titles.csv")

In [90]:
df.describe(include='all')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
count,8807,8807,8807,6173,7982,7976,8797,8807.000000,8803,8804,8807,8807
unique,8807,2,8807,4528,7692,748,1767,NaN,17,220,514,8775
top,s8807,Movie,Zubaan,Rajiv Chilaka,David Attenborough,United States,"January 1, 2020",NaN,TV-MA,1 Season,"Dramas, International Movies","Paranormal activity at a lush, abandoned prope..."
freq,1,6131,1,19,19,2818,109,NaN,3207,1793,362,4
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.180198,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.819312,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1925.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.000000,NaN,NaN,NaN,NaN


In [91]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [92]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


to find the duplicated in primary key

In [94]:
df[df['show_id'].duplicated()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


In [95]:
df['duration_minutes'] = df['duration'].where (df['duration'].str.contains(' min')).str.replace(' min','', regex= False).astype(float)

In [96]:
df['num_seasons'] = df['duration'].where(df['duration'].str.contains(' Seasons')).str.replace(' Seasons','',regex=False ).astype(float)

In [97]:
df.drop(columns= ['duration_minutes','num_seasons'], inplace=True)

### duration column cleaning

In [98]:
df['duration_value'] = df['duration'].str.extract(r'(\d+)').astype(float)
df['duration_unit'] = df['duration'].str.extract(r'(min|Seasons|Season)')

In [99]:
df['duration_time'] = df['duration_value'].where(df['duration_unit'] == 'min')
df['duration_season'] = df['duration_value'].where(df['duration_unit'].isin(['Season', 'Seasons']))

In [100]:
df['release_year'] = pd.to_numeric(df['release_year'],errors='coerce').astype(int)
df['release_year'].dtype

dtype('int64')

### changing the values of columns 'title','director','cast','country' to lower case

In [101]:
#df[['title','director','cast','country']].str.lower()

cols = ['title','director','cast','country']

df[cols] = df[cols].apply(lambda x :x.astype(str).str.lower())


#### split the comma seperated values into list

In [102]:
df['cast_list'] = df['cast'].str.split(', ')

# Normalization

#### 1.creating new dataframe/ table for cast 

In [103]:
casting_table = df[['title','cast']]
casting_table['cast'] = casting_table['cast'].str.split(', ')
casting_table = casting_table.explode('cast')
casting_table.head(5)

C:\Users\vinit\AppData\Local\Temp\ipykernel_25508\3522218473.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  casting_table['cast'] = casting_table['cast'].str.split(', ')


,title,cast
0,dick johnson is dead,nan
1,blood & water,ama qamata
1,blood & water,khosi ngema
1,blood & water,gail mabalane
1,blood & water,thabang molaba


#### 2.creating new dataframe.table for listed_in

In [104]:
listed_table = df[['title','listed_in']]
listed_table['listed_in'] = df['listed_in'].str.split(', ')
listed_table = listed_table.explode('listed_in')
listed_table.head(5)

C:\Users\vinit\AppData\Local\Temp\ipykernel_25508\2675414227.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listed_table['listed_in'] = df['listed_in'].str.split(', ')


,title,listed_in
0,dick johnson is dead,Documentaries
1,blood & water,International TV Shows
1,blood & water,TV Dramas
1,blood & water,TV Mysteries
2,ganglands,Crime TV Shows


#### 3. creating new dataframe/rable for country

In [105]:
country_table= df[['title','country']]
country_table['country'] = country_table['country'].str.split(', ')
country_table = country_table.explode('country')
country_table.head(110)

C:\Users\vinit\AppData\Local\Temp\ipykernel_25508\3394936898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_table['country'] = country_table['country'].str.split(', ')


,title,country
0,dick johnson is dead,united states
1,blood & water,south africa
2,ganglands,nan
3,jailbirds new orleans,nan
4,kota factory,india
...,...,...
92,into the night,belgium
93,jj+e,nan
94,show dogs,united kingdom
94,show dogs,united states


### date added and release_year analomolies detection

In [106]:
df.isnull().sum()

show_id               0
type                  0
title                 0
director              0
cast                  0
country               0
date_added           10
release_year          0
rating                4
duration              3
listed_in             0
description           0
duration_value        3
duration_unit         3
duration_time      2679
duration_season    6131
cast_list             0
dtype: int64

In [121]:
df['date_added'] = pd.to_datetime(df['date_added'].astype(str).str.strip(),errors='coerce')

In [ ]:
df[df['date_added'].isnull()]

In [108]:
df['added_year'] = df['date_added'].dt.year

invalid_dates = df[df['added_year'] < df['release_year']]

#problematic rows
invalid_dates.shape[0]
invalid_dates[['title','show_id', 'release_year','added_year']]


,title,show_id,release_year,added_year
1551,hilda,s1552,2021,2020.0
1696,polly pocket,s1697,2021,2020.0
2920,love is blind,s2921,2021,2020.0
3168,fuller house,s3169,2020,2019.0
3287,maradona in mexico,s3288,2020,2019.0
3369,bojack horseman,s3370,2020,2019.0
3433,the hook up plan,s3434,2020,2019.0
4844,unbreakable kimmy schmidt,s4845,2019,2018.0
4845,arrested development,s4846,2019,2018.0
5394,hans teeuwen: real rancour,s5395,2018,2017.0


In [109]:
df['inconsistent_dates'] = df['release_year'] > df['added_year']

#### Value correction in rating column

In [110]:
df['rating'].value_counts()

rating
TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
74 min         1
84 min         1
66 min         1
Name: count, dtype: int64

In [111]:
df['rating'] = df['rating'].str.strip().str.upper()

In [112]:
df['rating'] = df['rating'].replace(['NR', 'UNRATED', 'NOT RATED', 'UR'],'UNRATED')

In [113]:
#df.loc[~df['rating'].isin(valid_ratings), 'rating'] = 'UNRATED'

In [114]:
invalid_rating = df['rating'].str.contains('MIN', na=False)
df.loc[invalid_rating, 'rating'] = 'UNRATED'

In [115]:
df['rating'].value_counts()

rating
TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
UNRATED       86
G             41
TV-Y7-FV       6
NC-17          3
Name: count, dtype: int64

In [116]:
df['country'].value_counts()

country
united states                                    2818
india                                             972
nan                                               831
united kingdom                                    419
japan                                             245
                                                 ... 
russia, spain                                       1
croatia, slovenia, serbia, montenegro               1
japan, canada                                       1
united states, france, south korea, indonesia       1
canada, mexico, germany, south africa               1
Name: count, Length: 749, dtype: int64

# cleaning country column

In [117]:
df.isnull().sum()

show_id                  0
type                     0
title                    0
director                 0
cast                     0
country                  0
date_added              10
release_year             0
rating                   4
duration                 3
listed_in                0
description              0
duration_value           3
duration_unit            3
duration_time         2679
duration_season       6131
cast_list                0
added_year              10
inconsistent_dates       0
dtype: int64

In [118]:
df.isnull().sum()

show_id                  0
type                     0
title                    0
director                 0
cast                     0
country                  0
date_added              10
release_year             0
rating                   4
duration                 3
listed_in                0
description              0
duration_value           3
duration_unit            3
duration_time         2679
duration_season       6131
cast_list                0
added_year              10
inconsistent_dates       0
dtype: int64

In [125]:
df.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_value,duration_unit,duration_time,duration_season,cast_list,added_year,inconsistent_dates
0,s1,Movie,dick johnson is dead,kirsten johnson,nan,united states,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",90.0,min,90.0,NaN,[nan],2021.0,False
1,s2,TV Show,blood & water,nan,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2.0,Seasons,NaN,2.0,"[ama qamata, khosi ngema, gail mabalane, thaba...",2021.0,False
2,s3,TV Show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",nan,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1.0,Season,NaN,1.0,"[sami bouajila, tracy gotoas, samuel jouy, nab...",2021.0,False


In [127]:
df.drop(columns=['duration','duration_value','duration_unit','cast_list','listed_in','country'], inplace= True)

In [128]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'date_added',
       'release_year', 'rating', 'description', 'duration_time',
       'duration_season', 'added_year', 'inconsistent_dates'],
      dtype='object')

In [129]:
from sqlalchemy import create_engine
import urllib

#step1: connect to postgresql

username = "postgres"
raw_password = "#@giZHini0301"
host = "localhost"
port = "5432"
database = "sql_projects"
password = urllib.parse.quote_plus(raw_password) 
#print(repr(host))


#engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")
engine = create_engine(
    f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
)




#step 2 load dataframe into postgresql

#table_name = "netflix_shows"
#df.to_sql(table_name, engine, if_exists="replace", index= False)

table_names = {
    'netflix_show':df,
    'country':country_table,
    'casting':casting_table,
    'genre': listed_table
}

for table_name, dataframe in table_names.items():
    dataframe.to_sql(table_name, engine, if_exists="replace", index= False)
    print(f"data successfully laded into table '{table_name}' in database '{database}'.")



data successfully laded into table 'netflix_show' in database 'sql_projects'.
data successfully laded into table 'country' in database 'sql_projects'.
data successfully laded into table 'casting' in database 'sql_projects'.
data successfully laded into table 'genre' in database 'sql_projects'.
